# WildGuard Exploration Notebook

This notebook explores the WildChat detection results to identify patterns, prevalence rates, and interesting findings.

In [ ]:
import sys
sys.path.insert(0, '..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

from src.utils import load_jsonl, load_json
from src.config import OUTPUTS_DIR, FIGURES_DIR, DARK_PATTERN_CATEGORIES

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

## 1. Load Detection Results

In [ ]:
# Load WildChat detections
detections = load_jsonl(OUTPUTS_DIR / 'wildchat_detections.jsonl')
df = pd.DataFrame(detections)

print(f'Total detections: {len(df)}')
df.head()

## 2. Overall Prevalence

In [ ]:
# Calculate prevalence by category
category_counts = df['predicted_category'].value_counts()
total = len(df)

prevalence = pd.DataFrame({
    'Category': category_counts.index,
    'Count': category_counts.values,
    'Rate': category_counts.values / total,
    'Per 1000': 1000 * category_counts.values / total
})

print('\nPrevalence by Category:')
prevalence

In [ ]:
# Plot prevalence (excluding 'none')
flagged_prev = prevalence[prevalence['Category'] != 'none'].sort_values('Per 1000', ascending=True)

fig, ax = plt.subplots(figsize=(10, 6))
ax.barh(flagged_prev['Category'], flagged_prev['Per 1000'], color=sns.color_palette('husl', len(flagged_prev)))
ax.set_xlabel('Prevalence (per 1,000 turns)')
ax.set_title('Dark Pattern Prevalence by Category')
plt.tight_layout()
plt.savefig(FIGURES_DIR / 'prevalence_exploration.png', dpi=150)
plt.show()

## 3. Confidence Score Analysis

In [ ]:
# Overall confidence distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# All detections
axes[0].hist(df['predicted_confidence'], bins=20, edgecolor='black', alpha=0.7)
axes[0].set_xlabel('Confidence Score')
axes[0].set_ylabel('Count')
axes[0].set_title('All Detections')

# Flagged only (non-none)
flagged = df[df['predicted_category'] != 'none']
axes[1].hist(flagged['predicted_confidence'], bins=20, edgecolor='black', alpha=0.7, color='coral')
axes[1].set_xlabel('Confidence Score')
axes[1].set_ylabel('Count')
axes[1].set_title('Flagged Detections Only')

plt.tight_layout()
plt.savefig(FIGURES_DIR / 'confidence_exploration.png', dpi=150)
plt.show()

In [ ]:
# Confidence by category
fig, ax = plt.subplots(figsize=(12, 6))

categories = [c for c in DARK_PATTERN_CATEGORIES if c in df['predicted_category'].values]
conf_by_cat = [df[df['predicted_category'] == c]['predicted_confidence'].values for c in categories]

bp = ax.boxplot(conf_by_cat, labels=categories, patch_artist=True)
colors = sns.color_palette('husl', len(categories))
for patch, color in zip(bp['boxes'], colors):
    patch.set_facecolor(color)

ax.set_ylabel('Confidence Score')
ax.set_title('Confidence Distribution by Category')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.savefig(FIGURES_DIR / 'confidence_by_category.png', dpi=150)
plt.show()

## 4. Turn Index Analysis

In [ ]:
# Detection rate by turn index
if 'turn_index' in df.columns:
    turn_stats = df.groupby('turn_index').agg(
        total=('predicted_category', 'count'),
        flagged=('predicted_category', lambda x: (x != 'none').sum())
    ).reset_index()
    turn_stats['flag_rate'] = turn_stats['flagged'] / turn_stats['total']
    
    fig, ax = plt.subplots(figsize=(12, 5))
    ax.plot(turn_stats['turn_index'], turn_stats['flag_rate'], marker='o', linewidth=2, color='coral')
    ax.fill_between(turn_stats['turn_index'], turn_stats['flag_rate'], alpha=0.3, color='coral')
    ax.set_xlabel('Conversation Turn Index')
    ax.set_ylabel('Flag Rate')
    ax.set_title('Dark Pattern Detection Rate by Conversation Turn')
    plt.tight_layout()
    plt.savefig(FIGURES_DIR / 'flag_rate_by_turn.png', dpi=150)
    plt.show()
    
    print('\nTurn Index Statistics:')
    turn_stats.head(20)

## 5. Context Analysis

In [ ]:
# Analysis by model (if available)
if 'model' in df.columns:
    model_stats = df.groupby('model').agg(
        total=('predicted_category', 'count'),
        flagged=('predicted_category', lambda x: (x != 'none').sum())
    ).reset_index()
    model_stats['flag_rate'] = model_stats['flagged'] / model_stats['total']
    model_stats = model_stats.sort_values('flag_rate', ascending=False)
    
    print('Flag Rate by Model:')
    model_stats

## 6. High-Risk Detections

In [ ]:
# Find high-confidence detections
high_risk = df[(df['predicted_category'] != 'none') & (df['predicted_confidence'] > 0.7)]
high_risk = high_risk.sort_values('predicted_confidence', ascending=False)

print(f'High-risk detections (confidence > 0.7): {len(high_risk)}')
print(f'\nTop categories in high-risk:')
print(high_risk['predicted_category'].value_counts())

In [ ]:
# Sample high-risk content
print('\n=== Sample High-Risk Detections ===')
for i, row in high_risk.head(5).iterrows():
    print(f"\n--- {row['predicted_category']} (conf: {row['predicted_confidence']:.2f}) ---")
    content = row.get('content', 'N/A')
    print(content[:300] + '...' if len(content) > 300 else content)

## 7. Summary Statistics

In [ ]:
# Summary
print('=== WildGuard Exploration Summary ===')
print(f'\nTotal samples analyzed: {len(df)}')
print(f'Flagged samples: {len(df[df["predicted_category"] != "none"])}')
print(f'Overall flag rate: {len(df[df["predicted_category"] != "none"]) / len(df):.1%}')
print(f'\nMean confidence (flagged): {flagged["predicted_confidence"].mean():.2f}')
print(f'High-risk detections (>0.7 conf): {len(high_risk)}')

print('\n=== Key Findings ===')
top_cat = prevalence[prevalence['Category'] != 'none'].iloc[0]
print(f'- Most common pattern: {top_cat["Category"]} ({top_cat["Per 1000"]:.1f} per 1000 turns)')